In [ ]:
from deepgram import DeepgramClient, PrerecordedOptions, FileSource
import requests

import os
from dotenv import load_dotenv
import yt_dlp as youtube_dl

In [2]:
load_dotenv()

#mp4 directory
SAVE_DIR = '/'.join(os.getcwd().split('/')[:3]) + '/speechdir'
# Deepgram API key
DG_KEY = os.getenv('DEEPGRAM_API_KEY')

AUDIO_PATH = SAVE_DIR + '/audio_cont'


ydl_opts = {
  'format': 'best',
  'outtmpl': AUDIO_PATH + '/%(title)s.%(ext)s',
  'noplaylist': True,
  'extract-audio': True,
}

video = "https://www.youtube.com/watch?v=PeMlggyqz0Y"

NameError: name 'load_dotenv' is not defined

In [ ]:
def process_download(video_url: str = None, ydl_opts: dict = ydl_opts):
    #if in SAVE_PATH have another file, delete it to be empty
	if os.path.exists(AUDIO_PATH):
		for file in os.listdir(AUDIO_PATH):
			os.remove(AUDIO_PATH + '/' + file)
	with youtube_dl.YoutubeDL(ydl_opts) as ydl:
		info_dict = ydl.extract_info(video_url, download=True)
		video_url = info_dict.get("url", None)
		video_title = info_dict.get('title', None)
		video_length = info_dict.get('duration')
	return video_title, video_length

In [ ]:
video_title, video_length = process_download(video, ydl_opts)

In [ ]:
JSON_PATH = SAVE_DIR + '/json_cont'
# URL of the audio file
AUDIO_FILE = AUDIO_PATH + '/' + video_title + '.mp4'
# Path to save the transcript JSON file
TRANSCRIPT_FILE = JSON_PATH + '/' + video_title + '.json'

In [ ]:
def transcript():
    try:
        # STEP 1: Create a Deepgram client using the API key
        deepgram = DeepgramClient(DG_KEY)

        # Download the audio file from the URL
        if os.path.exists(JSON_PATH):
            for file in os.listdir(JSON_PATH):
                os.remove(JSON_PATH + '/' + file)

        with open(AUDIO_FILE, "rb") as file:
            buffer_data = file.read()

        payload: FileSource = {
            "buffer": buffer_data,
        }

        # STEP 2: Configure Deepgram options for audio analysis
        options = PrerecordedOptions(
            model="nova-2",
            smart_format=True,
        )

        # STEP 3: Call the transcribe_file method with the text payload and options
        response = deepgram.listen.prerecorded.v("1").transcribe_file(payload, options)

        # STEP 4: Write the response JSON to a file
        with open(TRANSCRIPT_FILE, "w") as transcript_file:
            transcript_file.write(response.to_json(indent=4))

        print("Transcript JSON file generated successfully.")

    except Exception as e:
        print(f"Exception: {e}")

In [ ]:
transcript()

In [ ]:
import json
def print_transcript(transcription_file: str = None):
  with open(transcription_file, "r") as file:
        data = json.load(file)
        result = data['results']['channels'][0]['alternatives'][0]['transcript']
        result = result.split('.')
        for sentence in result:
          print(sentence + '.')

In [ ]:
print_transcript(JSON_PATH + '/' + video_title + '.json')

In [ ]:
os.path.exists("Downloads/Machine Learning Explained in 100 Seconds.mp4")